# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

In [3]:
!kubectl label nodes apiserver-crd role=locust --overwrite

node "apiserver-crd" not labeled


## Start seldon-core

In [2]:
!helm install ../../helm-charts/seldon-core --name seldon-core --set grafana_prom_admin_password=password --set persistence.enabled=false --set cluster_manager.image.tag=0.3-SNAPSHOT --set apife.image.tag=0.1-SNAPSHOT --set engine.image.tag=0.2-SNAPSHOT

NAME:   seldon-core
LAST DEPLOYED: Tue Nov 28 11:21:42 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME                      TYPE       CLUSTER-IP  EXTERNAL-IP  PORT(S)                     AGE
alertmanager              ClusterIP  10.0.0.82   <none>       80/TCP                      6s
seldon-apiserver          NodePort   10.0.0.27   <none>       8080:30032/TCP              6s
seldon-cluster-manager    NodePort   10.0.0.106  <none>       8080:30030/TCP              6s
grafana-prom              NodePort   10.0.0.135  <none>       80:30034/TCP                6s
kafka                     NodePort   10.0.0.115  <none>       9092:30010/TCP              6s
prometheus-node-exporter  ClusterIP  None        <none>       9100/TCP                    5s
prometheus-seldon         ClusterIP  10.0.0.69   <none>       80/TCP                      5s
redis                     ClusterIP  10.0.0.190  <none>       6379/TCP                    5s
zookeeper-1               ClusterIP  10

In [3]:
!kubectl get deployments

NAME                      DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
alertmanager-deployment   1         1         1            1           26s
grafana-prom-deployment   1         1         1            1           26s
kafka                     1         1         1            1           25s
prometheus-deployment     1         1         1            1           25s
redis                     1         1         1            1           25s
seldon-apiserver          1         1         1            1           26s
seldon-cluster-manager    1         1         1            1           26s


# Integrating with Kubernetes API

In [4]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.5",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
         

## Create Seldon Deployment

In [5]:
!kubectl apply -f resources/model.json

seldondeployment "seldon-deployment-example" created


In [6]:
!kubectl get seldondeployments

NAME                        AGE
seldon-deployment-example   3s


In [7]:
!kubectl describe seldondeployments seldon-deployment-example 

Name:         seldon-deployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"SeldonDeployment","metadata":{"name":"seldon-deployment-example","namespace":"default","self...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         SeldonDeployment
Metadata:
  Cluster Name:        
  Creation Timestamp:  2017-11-28T11:23:39Z
  Generation:          0
  Initializers:        <nil>
  Resource Version:    1221485
  Self Link:           /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/seldondeployments/seldon-deployment-example
  UID:                 95102f7e-d42e-11e7-9b11-08002751d93c
Spec:
  Annotations:
    Project _ Name:  FX Market Prediction
  Endpoint:
    Service _ Host:  test-deployment
    Service _ Port:  8000
    Type:            REST
  Name:              test-deployment
  Oauth _ Key:       oauth-key
  Oauth _ Secret:    oauth-secret

In [8]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[replicasAvailable:1 name:test-deployment-fx-market-predictor replicas:1]]]

## Get predictions

In [9]:
%%bash
API_SERVER=35.195.94.148:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
#SERVER=0.0.0.0:8080
echo "$SERVER"
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

192.168.99.100:30032
{
  "status": {
    "code": 0,
    "info": "",
    "reason": "",
    "status": "SUCCESS"
  },
  "meta": {
    "puid": "jfgf1nr41vmjqlg4f4r312s8n3",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["class0", "class1", "class2"],
    "tensor": {
      "shape": [1, 3],
      "values": [0.1, 0.9, 0.5]
    }
  }
}

## Update deployment with canary

In [10]:
!cat resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.5",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
         

In [11]:
!kubectl apply -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" configured


In [13]:
!kubectl get seldondeployments mldeployment-example -o jsonpath='{.status}'

Error from server (NotFound): seldondeployments.machinelearning.seldon.io "mldeployment-example" not found


In [14]:
%%bash
API_SERVER=35.189.249.200:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

{
  "status": {
    "code": 0,
    "info": "",
    "reason": "",
    "status": "SUCCESS"
  },
  "meta": {
    "puid": "f0lnnvujc729jf9ggln9qrc82p",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["class0", "class1", "class2"],
    "tensor": {
      "shape": [1, 3],
      "values": [0.1, 0.9, 0.5]
    }
  }
}

## Load test

In [20]:
!helm install --set oauth.key=oauth-key --set oauth.secret=oauth-secret --name loadtest ../../helm-charts/seldon-core-loadtesting

NAME:   loadtest
LAST DEPLOYED: Sun Nov  5 14:49:42 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP  EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.0.0.26   <none>       5557:30376/TCP,5558:30483/TCP,8089:30169/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-2sd7k   0/1    ContainerCreating  0         0s
locust-master-1-6gf8s  0/1    ContainerCreating  0         0s




# Tear down

In [13]:
!kubectl delete -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" deleted


In [17]:
!helm delete loadtest --purge

Error: release: "loadtest" not found


In [1]:
!helm delete seldon-core --purge

release "seldon-core" deleted
